In [1]:
import pandas as pd
import numpy as np
weather_stations = pd.read_csv('/Users/evelyn/Desktop/weather-stations.csv', encoding= 'unicode_escape')


norodata = pd.read_excel('/Users/evelyn/Desktop/Cleaned data - Salmonella and norovirus.xlsx', sheet_name = 'Norovirus')
           

In [10]:
import re
import os #Import for setting working dirrectory
import requests #Import for downloading data from database
from tqdm import tqdm
import sys
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Base URL and filename
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/" #Daily data path 
station = pd.DataFrame()
for i in range(len(norodata)):
    print('{:.4%}'.format(i/len(norodata)))
    df = weather_stations[weather_stations['STATE'].isin([norodata.loc[i,"Exposurestate"]])]
    for j in range(0,11):
        if(pd.isna(norodata.loc[i,"ExposureCounty"+str(j+1)])):
            break
        else:
            noro_station = pd.DataFrame(norodata.iloc[i,:7]).T
            noro_station['County'] = norodata.loc[i,"ExposureCounty"+str(j+1)]
            new_stations = df[df.County.str.contains(str(norodata.loc[i,"ExposureCounty"+str(j+1)]),regex= True, na=False)]
            if(new_stations.empty):
                station = station.append(noro_station)
                station = station.reset_index(drop=True)
                continue
            else:
                #检查CSV file 完整性
                new_stations = new_stations.reset_index(drop=True)
                for z in range(len(new_stations)):
                    wban_unmodified = new_stations.loc[z, "WBAN"]
                    if 0 < wban_unmodified < 10000: 
                        new_stations.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)
                    usaf = new_stations.loc[z, "USAF"]
                    #print(usaf) 
                    wban = new_stations.loc[z, "WBAN"]
                    #print(wban)
                    startyear = str(noro_station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
                    endyear = str(noro_station.loc[i , "LastExposure"])[:4]  
                    #print(endyear)
                    filename_noaa = str(usaf) + str(wban) + ".csv" 
                    for yr in range(int(startyear), int(endyear)+1): 
                        url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                        #print(url_loop_daily)

                        #Identifies file on website
                        csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                        #Sets the file name of the local imported file
                        filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                        os.chdir('/Users/evelyn/Desktop/noro') #Sets path for daily data csv deposit  
                        open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
                        total = sum(1 for line in open(filename_local_daily))
                        if(total < 35):
                            new_stations.loc[z, "Incom"] = 'No'
                        else:
                            new_stations.loc[z, "Incom"] = 'Yes'
                            
                new_stations = new_stations[~(new_stations["Incom"] == 'No')]
                new_stations = new_stations.dropna(subset = ['USAF'])
                new_stations = new_stations.reset_index(drop = True)
                if(new_stations.empty):
                    station = station.append(noro_station)
                    station = station.reset_index(drop=True)
                else:
                    noro_newstation = noro_station.append([noro_station]*(len(new_stations)-1))
                    noro_newstation = noro_newstation.reset_index(drop = True)
                    for index in range(len(new_stations)):
                        noro_newstation.loc[index, "USAF"] = new_stations.loc[index, "USAF"]
                        noro_newstation.loc[index, "WBAN"] = new_stations.loc[index, "WBAN"]
                    station = station.append(noro_newstation)
                    station = station.reset_index(drop = True)
                        
                                               

0.0000%
0.0401%
0.0802%
0.1203%
0.1604%
0.2006%
0.2407%
0.2808%
0.3209%
0.3610%
0.4011%
0.4412%
0.4813%
0.5215%
0.5616%
0.6017%
0.6418%
0.6819%
0.7220%
0.7621%
0.8022%
0.8424%
0.8825%
0.9226%
0.9627%
1.0028%
1.0429%
1.0830%
1.1231%
1.1633%
1.2034%
1.2435%
1.2836%
1.3237%
1.3638%
1.4039%
1.4440%
1.4842%
1.5243%
1.5644%
1.6045%
1.6446%
1.6847%
1.7248%
1.7649%
1.8051%
1.8452%
1.8853%
1.9254%
1.9655%
2.0056%
2.0457%
2.0858%
2.1260%
2.1661%
2.2062%
2.2463%
2.2864%
2.3265%
2.3666%
2.4067%
2.4469%
2.4870%
2.5271%
2.5672%
2.6073%
2.6474%
2.6875%
2.7276%
2.7677%
2.8079%
2.8480%
2.8881%
2.9282%
2.9683%
3.0084%
3.0485%
3.0886%
3.1288%
3.1689%
3.2090%
3.2491%
3.2892%
3.3293%
3.3694%
3.4095%
3.4497%
3.4898%
3.5299%
3.5700%
3.6101%
3.6502%
3.6903%
3.7304%
3.7706%
3.8107%
3.8508%
3.8909%
3.9310%
3.9711%
4.0112%
4.0513%
4.0915%
4.1316%
4.1717%
4.2118%
4.2519%
4.2920%
4.3321%
4.3722%
4.4124%
4.4525%
4.4926%
4.5327%
4.5728%
4.6129%
4.6530%
4.6931%
4.7333%
4.7734%
4.8135%
4.8536%
4.8937%
4.9338%
4.9739%


In [11]:
station.isnull().sum()

CDCID                0
GenusName            0
InitialExposure      0
LastExposure         0
TotalCases           0
Exposurestate        0
ExposureState        0
County               0
USAF               289
WBAN               289
dtype: int64

In [14]:
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/" #Daily data path 
station["WBAN"] = pd.to_numeric(station["WBAN"], errors='coerce').fillna('0').astype('int32')
for z in range(len(station)):
    wban_unmodified = station.loc[z, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        
        station.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)
for i in range(len(station)):
    print('{:.4%}'.format(i/len(station)))
    if(pd.isna(station.loc[i, "USAF"])):
        continue
    else:
        usaf = station.loc[i, "USAF"]
                    #print(usaf) 
        wban = station.loc[i, "WBAN"]
                    #print(wban)
        startyear = str(station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
        endyear = str(station.loc[i , "LastExposure"])[:4]  
                    #print(endyear)
        filename_noaa = str(usaf) + str(wban) + ".csv" 
        for yr in range(int(startyear), int(endyear)+1): 
            
            url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                        #print(url_loop_daily)

                        #Identifies file on website
            csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                        #Sets the file name of the local imported file
            filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
            os.chdir('/Users/evelyn/Desktop/noro') #Sets path for daily data csv deposit  
            open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
        
        
    
    

0.0000%
0.0122%
0.0245%
0.0367%
0.0490%
0.0612%
0.0735%
0.0857%
0.0980%
0.1102%
0.1225%
0.1347%
0.1470%
0.1592%
0.1714%
0.1837%
0.1959%
0.2082%
0.2204%
0.2327%
0.2449%
0.2572%
0.2694%
0.2817%
0.2939%
0.3061%
0.3184%
0.3306%
0.3429%
0.3551%
0.3674%
0.3796%
0.3919%
0.4041%
0.4164%
0.4286%
0.4409%
0.4531%
0.4653%
0.4776%
0.4898%
0.5021%
0.5143%
0.5266%
0.5388%
0.5511%
0.5633%
0.5756%
0.5878%
0.6000%
0.6123%
0.6245%
0.6368%
0.6490%
0.6613%
0.6735%
0.6858%
0.6980%
0.7103%
0.7225%
0.7348%
0.7470%
0.7592%
0.7715%
0.7837%
0.7960%
0.8082%
0.8205%
0.8327%
0.8450%
0.8572%
0.8695%
0.8817%
0.8940%
0.9062%
0.9184%
0.9307%
0.9429%
0.9552%
0.9674%
0.9797%
0.9919%
1.0042%
1.0164%
1.0287%
1.0409%
1.0531%
1.0654%
1.0776%
1.0899%
1.1021%
1.1144%
1.1266%
1.1389%
1.1511%
1.1634%
1.1756%
1.1879%
1.2001%
1.2123%
1.2246%
1.2368%
1.2491%
1.2613%
1.2736%
1.2858%
1.2981%
1.3103%
1.3226%
1.3348%
1.3470%
1.3593%
1.3715%
1.3838%
1.3960%
1.4083%
1.4205%
1.4328%
1.4450%
1.4573%
1.4695%
1.4818%
1.4940%
1.5062%
1.5185%


In [23]:
station["WBAN"] = pd.to_numeric(station["WBAN"], errors='coerce').fillna('0').astype('int32')
for a in range(len(station)):
    wban_unmodified = station.loc[a, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        station.loc[a, "WBAN"] = str(wban_unmodified).zfill(5)

# tackle the SNDP and FRSHTT information in matched counties
path = '/Users/evelyn/Desktop/noro/'
store_path = '/Users/evelyn/Desktop/noro-weather/'
for i in range(len(station)):
    print('{:.4%}'.format(i/len(station)))
    if(pd.isna(station.loc[i, "USAF"])):
        continue
    else:
        usaf = station.loc[i, "USAF"]
                    #print(usaf) 
        wban = station.loc[i, "WBAN"]
        startyear = str(station.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                    #print(startyear)
        endyear = str(station.loc[i , "LastExposure"])[:4]  
        for yr in range(int(startyear), int(endyear)+1): 
            filename = str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
            df = pd.read_csv(path + filename)
            for j in range(len(df)):
                if not 'SNDP' in df.columns:
                    break
                else:
                    if(df.loc[j, "SNDP"] > 999):
                        df.loc[j, "SNDP"] = 0
                    if((df.loc[j, "FRSHTT"]%10) > 0):# 个位
                        df.loc[j, "Tornado or Funnel Cloud"] = int(1)
                    else:
                        df.loc[j, "Tornado or Funnel Cloud"] = int(0)
                    if(df.loc[j, "FRSHTT"]//10%10 > 0):#十位
                        df.loc[j, "Thunder"] = int(1)
                    else:
                        df.loc[j, "Thunder"] = int(0)
                    if((df.loc[j, "FRSHTT"]//100%10) > 0):#百位
                        df.loc[j, "Hail"] = int(1)
                    else:
                        df.loc[j, "Hail"] = int(0)
                    if((df.loc[j, "FRSHTT"]//1000%10) > 0):#千位
                        df.loc[j, "Snow or Ice Pellets"] = int(1)
                    else:
                        df.loc[j, "Snow or Ice Pellets"] = int(0)
                    if((df.loc[j, "FRSHTT"]//10000 % 10) > 0):
                        df.loc[j, "Rain or Drizzle"] = int(1)
                    else:
                        df.loc[j, "Rain or Drizzle"] = int(0)
                    if((df.loc[j, "FRSHTT"]//100000 % 10) > 0):
                        df.loc[j, "Fog"] = int(1)
                    else:
                        df.loc[j, "Fog"] = int(0)

            df.to_csv(store_path + filename)

0.0000%
0.0122%
0.0245%
0.0367%
0.0490%
0.0612%
0.0735%
0.0857%
0.0980%
0.1102%
0.1225%
0.1347%
0.1470%
0.1592%
0.1714%
0.1837%
0.1959%
0.2082%
0.2204%
0.2327%
0.2449%
0.2572%
0.2694%
0.2817%
0.2939%
0.3061%
0.3184%
0.3306%
0.3429%
0.3551%
0.3674%
0.3796%
0.3919%
0.4041%
0.4164%
0.4286%
0.4409%
0.4531%
0.4653%
0.4776%
0.4898%
0.5021%
0.5143%
0.5266%
0.5388%
0.5511%
0.5633%
0.5756%
0.5878%
0.6000%
0.6123%
0.6245%
0.6368%
0.6490%
0.6613%
0.6735%
0.6858%
0.6980%
0.7103%
0.7225%
0.7348%
0.7470%
0.7592%
0.7715%
0.7837%
0.7960%
0.8082%
0.8205%
0.8327%
0.8450%
0.8572%
0.8695%
0.8817%
0.8940%
0.9062%
0.9184%
0.9307%
0.9429%
0.9552%
0.9674%
0.9797%
0.9919%
1.0042%
1.0164%
1.0287%
1.0409%
1.0531%
1.0654%
1.0776%
1.0899%
1.1021%
1.1144%
1.1266%
1.1389%
1.1511%
1.1634%
1.1756%
1.1879%
1.2001%
1.2123%
1.2246%
1.2368%
1.2491%
1.2613%
1.2736%
1.2858%
1.2981%
1.3103%
1.3226%
1.3348%
1.3470%
1.3593%
1.3715%
1.3838%
1.3960%
1.4083%
1.4205%
1.4328%
1.4450%
1.4573%
1.4695%
1.4818%
1.4940%
1.5062%
1.5185%


In [24]:
station.to_csv('/Users/evelyn/Desktop/new-/noro_station.csv')

In [13]:
noronull = station[station["USAF"].isnull()]
noronull.to_csv('/Users/evelyn/Desktop/noronull.csv')

In [25]:
noro_null = pd.read_csv('/Users/evelyn/Desktop/noronull.csv')

In [26]:
noro_null

,CDCID,GenusName,InitialExposure,LastExposure,TotalCases,Exposurestate,ExposureState,County,County1,LAT,LON
0,46,Norovirus,2009/1/1,2009/1/2,6,IL,Illinois,McHenry,McHenry County,42.324298,-88.452245
1,80,Norovirus,2009/3/5,2009/3/7,9,TN,Tennessee,Bradley,Bradley County,35.153914,-84.859414
2,1969,Norovirus,2009/5/31,2009/5/31,15,OH,Ohio,Medina,Medina County,41.116174,-81.899764
3,2465,Norovirus,2009/7/13,2009/7/13,31,NH,New Hampshire,Sullivan,Sullivan County,43.361188,-72.222084
4,2340,Norovirus,2009/6/20,2009/6/20,13,KS,Kansas,Franklin,Franklin County,38.558019,-95.278962
...,...,...,...,...,...,...,...,...,...,...,...
283,298726,Norovirus,2021/3/8,2021/3/8,13,MA,Massachusetts,Hampshire,Hampshire County,42.339459,-72.663694
284,298877,Norovirus,2021/12/19,2021/12/19,8,OH,Ohio,Coshocton,Coshocton County,40.296710,-81.930109
285,298885,Norovirus,2021/12/23,2021/12/27,10,MN,Minnesota,Washington,Washington County,45.037929,-92.890117
286,298887,Norovirus,2021/12/25,2021/12/26,8,VA,Virginia,Lynchburg,Lynchburg city,37.399016,-79.195458


In [27]:
from math import radians, cos, sin, asin, sqrt, atan
from math import *
def haversine(lat1, lon1, lat2, lon2):
    
    R = 3959.87433 #Earth radius in miles
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + sin(dlon/2)**2 * cos(lat1) * cos(lat2)
    c = 2 * asin(sqrt(a))
    return R*c          

In [29]:
import heapq
url_base = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/"
weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').fillna('0').astype('int32')
for z in range(len(weather_stations)):
    wban_unmodified = weather_stations.loc[z, "WBAN"]
    if 0 < wban_unmodified < 10000: 
        
        weather_stations.loc[z, "WBAN"] = str(wban_unmodified).zfill(5)

for i in range(len(noro_null)):
    print('{:.4%}'.format(i/len(noro_null)))
    slat = float(noro_null.loc[i, "LAT"])
    slon = float(noro_null.loc[i, "LON"])
    for j in range(len(weather_stations)):
        lat = float(weather_stations.loc[j, "LAT"])
        lon = float(weather_stations.loc[j, "LON"])
        weather_stations.loc[j, "DIS"] = haversine(slat, slon, lat, lon)
    dis = weather_stations["DIS"].tolist()
    
    weather_stations["WBAN"] = pd.to_numeric(weather_stations["WBAN"], errors='coerce').astype('int32')
    for l in range(60):
        z = []
        min_index = list(map(dis.index, heapq.nsmallest(3, dis)))
        dlat = [weather_stations.loc[min_index[0], "LAT"], weather_stations.loc[min_index[1], "LAT"], weather_stations.loc[min_index[2],"LAT"]]
        dlon = [weather_stations.loc[min_index[0], "LON"], weather_stations.loc[min_index[1], "LON"], weather_stations.loc[min_index[2],"LON"]]
        if(min(dlat) < slat and slat < max(dlat) and min(dlon) < slon and slon < max(dlon)):
            for m in range(3):
                usaf = weather_stations.loc[min_index[m], "USAF"]
                #print(usaf) 
                wban = weather_stations.loc[min_index[m], "WBAN"]
                #print(wban)
                startyear = str(noro_null.loc[i , "InitialExposure"])[:4] #Extracts the year from the date string by taking the first four elements
                #print(startyear)
                endyear = str(noro_null.loc[i , "LastExposure"])[:4]  
                #print(endyear)
                filename_noaa = str(usaf) + str(wban) + ".csv" 
                for yr in range(int(startyear), int(endyear)+1): 
                    url_loop_daily = url_base + str(yr) + "/" + filename_noaa
                    #print(url_loop_daily)

                    #Identifies file on website
                    csv_current_daily = requests.get(url_loop_daily, allow_redirects = True)

                    #Sets the file name of the local imported file
                    filename_local_daily =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                    os.chdir('/Users/evelyn/Desktop/noro_null') #Sets path for daily data csv deposit  
                    open(filename_local_daily, "wb").write(csv_current_daily.content) #Write a csv for daily data
                    total = sum(1 for line in open(filename_local_daily))
                    if(total < 30):
                        dis[min_index[m]] = 100
                        z.append(100)
                    else:
                        z.append(0)
            if(max(z) < 50):
                break
            else: 
                continue
        else:
            dis[min_index[2]] = 100
    
    for k in range(3):
        index = min_index[k]
        noro_null.loc[i, "USAF"+str(k)] = weather_stations.loc[index, "USAF"]
        noro_null.loc[i, "WBAN"+str(k)] = weather_stations.loc[index, "WBAN"]
        noro_null.loc[i, "DIS"+str(k)] = dis[index]
    weather_stations = weather_stations.drop(columns = ["DIS"])
    

0.0000%
0.3472%
0.6944%
1.0417%
1.3889%
1.7361%
2.0833%
2.4306%
2.7778%
3.1250%
3.4722%
3.8194%
4.1667%
4.5139%
4.8611%
5.2083%
5.5556%
5.9028%
6.2500%
6.5972%
6.9444%
7.2917%
7.6389%
7.9861%
8.3333%
8.6806%
9.0278%
9.3750%
9.7222%
10.0694%
10.4167%
10.7639%
11.1111%
11.4583%
11.8056%
12.1528%
12.5000%
12.8472%
13.1944%
13.5417%
13.8889%
14.2361%
14.5833%
14.9306%
15.2778%
15.6250%
15.9722%
16.3194%
16.6667%
17.0139%
17.3611%
17.7083%
18.0556%
18.4028%
18.7500%
19.0972%
19.4444%
19.7917%
20.1389%
20.4861%
20.8333%
21.1806%
21.5278%
21.8750%
22.2222%
22.5694%
22.9167%
23.2639%
23.6111%
23.9583%
24.3056%
24.6528%
25.0000%
25.3472%
25.6944%
26.0417%
26.3889%
26.7361%
27.0833%
27.4306%
27.7778%
28.1250%
28.4722%
28.8194%
29.1667%
29.5139%
29.8611%
30.2083%
30.5556%
30.9028%
31.2500%
31.5972%
31.9444%
32.2917%
32.6389%
32.9861%
33.3333%
33.6806%
34.0278%
34.3750%
34.7222%
35.0694%
35.4167%
35.7639%
36.1111%
36.4583%
36.8056%
37.1528%
37.5000%
37.8472%
38.1944%
38.5417%
38.8889%
39.2361%
39.

In [30]:
for i in range(len(noro_null)):
    noro_null.loc[i, "W0"] = 1/(1/noro_null.loc[i, "DIS0"] + 1/noro_null.loc[i, "DIS1"] + 1/noro_null.loc[i, "DIS2"])/1/noro_null.loc[i, "DIS0"]
    noro_null.loc[i, "W1"] = 1/(1/noro_null.loc[i, "DIS0"] + 1/noro_null.loc[i, "DIS1"] + 1/noro_null.loc[i, "DIS2"])/1/noro_null.loc[i, "DIS1"]
    noro_null.loc[i, "W2"] = 1/(1/noro_null.loc[i, "DIS0"] + 1/noro_null.loc[i, "DIS1"] + 1/noro_null.loc[i, "DIS2"])/1/noro_null.loc[i, "DIS2"]
    
    

In [32]:
noro_null.to_csv('/Users/evelyn/Desktop/new-/noro_null.csv')

In [33]:
noro_fnull = pd.read_csv('/Users/evelyn/Desktop/new-/noro_null.csv')

In [34]:
import numpy as np
path = '/Users/evelyn/Desktop/noro_null/'
store_path = '/Users/evelyn/Desktop/noro_fnull/'
variable = ["TEMP", "DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN","PRCP"]
for i in range(len(noro_fnull)):
    print('{:.4%}'.format(i/len(noro_fnull)))
    if(pd.isna(noro_fnull.loc[i, "USAF2"])):
        continue
    else:
        file = []
        startyear = str(noro_fnull.loc[i , "InitialExposure"])[:4]
        endyear = str(noro_fnull.loc[i , "LastExposure"])[:4]
        w = [float(noro_fnull.loc[i, "W0"]), float(noro_fnull.loc[i, "W1"]), float(noro_fnull.loc[i, "W2"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(3):
                usaf = noro_fnull.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(noro_fnull.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            df2 = pd.read_csv(path + file[2])
            for j in range(len(df0)):
                date_string = df0.loc[j, "DATE"]
                if not(df1.DATE.str.contains(date_string).any() and df2.DATE.str.contains(date_string).any()):
                    df0 = df0.drop(index = [j])
            df0 = df0.reset_index(drop = True)
            for n in range(len(df0)):
                index1 = df1[df1.DATE.str.contains(df0.loc[n, "DATE"])].index.tolist()[0]
                index2 = df2[df2.DATE == df0.loc[n, "DATE"]].index.tolist()[0]
                for vr in range(len(variable)):
                    var = [float(df0.loc[n, variable[vr]]), float(df1.loc[index1, variable[vr]]), float(df2.loc[index2, variable[vr]])]
                    df0.loc[n, variable[vr]] = round(sum(np.multiply(var,w)), 1)
                #更改sndp
                sndp = [df0.loc[n, "SNDP"], df1.loc[index1, "SNDP"], df2.loc[index2, "SNDP"]]
                for sn in range(3):
                    if(sndp[sn] > 999):
                        sndp[sn] = 0
                df0.loc[n, "SNDP"] = round(sum(np.multiply(sndp,w)), 1)
                #对FRSHTT进行修改
                fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"], df2.loc[index2, "FRSHTT"]]
                if(((fr[0]%10) + (fr[1]%10) + (fr[2]%10)) > 1):# 个位
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                else:
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                if(((fr[0]//10%10) + (fr[1]//10%10) + (fr[2]//10%10)) > 1):#十位
                    df0.loc[n, "Thunder"] = int(1)
                else:
                    df0.loc[n, "Thunder"] = int(0)
                if(((fr[0]//100%10) + (fr[1]//100%10) + (fr[2]//100%10)) > 1):#百位
                    df0.loc[n, "Hail"] = int(1)
                else:
                    df0.loc[n, "Hail"] = int(0)
                if(((fr[0]//1000%10) + (fr[1]//1000%10) + (fr[2]//1000%10)) > 1):#千位
                    df0.loc[n, "Snow or Ice Pellets"] = int(1)
                else:
                    df0.loc[n, "Snow or Ice Pellets"] = int(0)
                if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10) + (fr[2]//10000 % 10)) > 1):
                    df0.loc[n, "Rain or Drizzle"] = int(1)
                else:
                    df0.loc[n, "Rain or Drizzle"] = int(0)
                if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10) + (fr[2]//100000 % 10)) > 1):
                    df0.loc[n, "Fog"] = int(1)
                else:
                    df0.loc[n, "Fog"] = int(0)
            
            df0 = df0.drop(columns = ["FRSHTT"])      
            new_usaf = noro_fnull.loc[i, "USAF0"]
            new_wban = pd.to_numeric(noro_fnull.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)
                    

0.0000%
0.3472%
0.6944%
1.0417%
1.3889%
1.7361%
2.0833%
2.4306%
2.7778%
3.1250%
3.4722%
3.8194%
4.1667%
4.5139%
4.8611%
5.2083%
5.5556%
5.9028%
6.2500%
6.5972%
6.9444%
7.2917%
7.6389%
7.9861%
8.3333%
8.6806%
9.0278%
9.3750%
9.7222%
10.0694%
10.4167%
10.7639%
11.1111%
11.4583%
11.8056%
12.1528%
12.5000%
12.8472%
13.1944%
13.5417%
13.8889%
14.2361%
14.5833%
14.9306%
15.2778%
15.6250%
15.9722%
16.3194%
16.6667%
17.0139%
17.3611%
17.7083%
18.0556%
18.4028%
18.7500%
19.0972%
19.4444%
19.7917%
20.1389%
20.4861%
20.8333%
21.1806%
21.5278%
21.8750%
22.2222%
22.5694%
22.9167%
23.2639%
23.6111%
23.9583%
24.3056%
24.6528%
25.0000%
25.3472%
25.6944%
26.0417%
26.3889%
26.7361%
27.0833%
27.4306%
27.7778%
28.1250%
28.4722%
28.8194%
29.1667%
29.5139%
29.8611%
30.2083%
30.5556%
30.9028%
31.2500%
31.5972%
31.9444%
32.2917%
32.6389%
32.9861%
33.3333%
33.6806%
34.0278%
34.3750%
34.7222%
35.0694%
35.4167%
35.7639%
36.1111%
36.4583%
36.8056%
37.1528%
37.5000%
37.8472%
38.1944%
38.5417%
38.8889%
39.2361%
39.

In [35]:
#two county intepolate
import numpy as np
path = '/Users/evelyn/Desktop/noro_null/'
store_path = '/Users/evelyn/Desktop/noro_fnull/'
variable = ["TEMP", "DEWP", "SLP", "STP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN","PRCP"]
for i in range(len(noro_fnull)):
    print('{:.4%}'.format(i/len(noro_fnull)))
    if(pd.isna(noro_fnull.loc[i, "USAF2"]) and pd.notna(noro_fnull.loc[i, "USAF1"])):
        file = []
        startyear = str(noro_fnull.loc[i , "InitialExposure"])[:4]
        endyear = str(noro_fnull.loc[i , "LastExposure"])[:4]
        w = [float(noro_fnull.loc[i, "W0"]), float(noro_fnull.loc[i, "W1"])]
        for yr in range(int(startyear), int(endyear)+1):
            for k in range(2):
                usaf = noro_fnull.loc[i, "USAF"+str(k)]
                wban = pd.to_numeric(noro_fnull.loc[i, "WBAN"+str(k)]).astype('int32')
                filename =  str(usaf) + str(wban) + "-" + str(yr) + "DAILY.csv" 
                file.append(filename)
            df0 = pd.read_csv(path + file[0])
            df1 = pd.read_csv(path + file[1])
            for j in range(len(df0)):
                date_string = df0.loc[j, "DATE"]
                if not(df1.DATE.str.contains(date_string).any()):
                    df0 = df0.drop(index = [j])
            df0 = df0.reset_index(drop = True)
            for n in range(len(df0)):
                index1 = df1[df1.DATE.str.contains(df0.loc[n, "DATE"])].index.tolist()[0]
                for vr in range(len(variable)):
                    var = [float(df0.loc[n, variable[vr]]), float(df1.loc[index1, variable[vr]])]
                    df0.loc[n, variable[vr]] = round(sum(np.multiply(var,w)), 1)
                #更改sndp
                sndp = [df0.loc[n, "SNDP"], df1.loc[index1, "SNDP"]]
                for sn in range(2):
                    if(sndp[sn] > 999):
                        sndp[sn] = 0
                df0.loc[n, "SNDP"] = round(sum(np.multiply(sndp,w)), 1)
                #对FRSHTT进行修改
                fr = [df0.loc[n, "FRSHTT"], df1.loc[index1, "FRSHTT"]]
                if(((fr[0]%10) + (fr[1]%10)) > 1):# 个位
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(1)
                else:
                    df0.loc[n, "Tornado or Funnel Cloud"] = int(0)
                if(((fr[0]//10%10) + (fr[1]//10%10)) > 1):#十位
                    df0.loc[n, "Thunder"] = int(1)
                else:
                    df0.loc[n, "Thunder"] = int(0)
                if(((fr[0]//100%10) + (fr[1]//100%10)) > 1):#百位
                    df0.loc[n, "Hail"] = int(1)
                else:
                    df0.loc[n, "Hail"] = int(0)
                if(((fr[0]//1000%10) + (fr[1]//1000%10))  > 1):#千位
                    df0.loc[n, "Snow or Ice Pellets"] = int(1)
                else:
                    df0.loc[n, "Snow or Ice Pellets"] = int(0)
                if(((fr[0]//10000 % 10) + (fr[1]//10000 % 10)) > 1):
                    df0.loc[n, "Rain or Drizzle"] = int(1)
                else:
                    df0.loc[n, "Rain or Drizzle"] = int(0)
                if(((fr[0]//100000 % 10) + (fr[1]//100000 % 10)) > 1):
                    df0.loc[n, "Fog"] = int(1)
                else:
                    df0.loc[n, "Fog"] = int(0)
            
            df0 = df0.drop(columns = ["FRSHTT"])      
            new_usaf = pd.to_numeric(noro_fnull.loc[i, "USAF0"]).astype('int32')
            new_wban = pd.to_numeric(noro_fnull.loc[i, "WBAN0"]).astype('int32')
            new_filename = str(new_usaf)+str(new_wban) + "-" + str(yr) + "DAILY.csv"
            df0.to_csv(store_path + new_filename)
    else:
                   continue
                

0.0000%
0.3472%
0.6944%
1.0417%
1.3889%
1.7361%
2.0833%
2.4306%
2.7778%
3.1250%
3.4722%
3.8194%
4.1667%
4.5139%
4.8611%
5.2083%
5.5556%
5.9028%
6.2500%
6.5972%
6.9444%
7.2917%
7.6389%
7.9861%
8.3333%
8.6806%
9.0278%
9.3750%
9.7222%
10.0694%
10.4167%
10.7639%
11.1111%
11.4583%
11.8056%
12.1528%
12.5000%
12.8472%
13.1944%
13.5417%
13.8889%
14.2361%
14.5833%
14.9306%
15.2778%
15.6250%
15.9722%
16.3194%
16.6667%
17.0139%
17.3611%
17.7083%
18.0556%
18.4028%
18.7500%
19.0972%
19.4444%
19.7917%
20.1389%
20.4861%
20.8333%
21.1806%
21.5278%
21.8750%
22.2222%
22.5694%
22.9167%
23.2639%
23.6111%
23.9583%
24.3056%
24.6528%
25.0000%
25.3472%
25.6944%
26.0417%
26.3889%
26.7361%
27.0833%
27.4306%
27.7778%
28.1250%
28.4722%
28.8194%
29.1667%
29.5139%
29.8611%
30.2083%
30.5556%
30.9028%
31.2500%
31.5972%
31.9444%
32.2917%
32.6389%
32.9861%
33.3333%
33.6806%
34.0278%
34.3750%
34.7222%
35.0694%
35.4167%
35.7639%
36.1111%
36.4583%
36.8056%
37.1528%
37.5000%
37.8472%
38.1944%
38.5417%
38.8889%
39.2361%
39.

FileNotFoundError: [Errno 2] No such file or directory: '/Users/evelyn/Desktop/noro_null/72788499999-2015DAILY.csv'